## Fake News Classifier Using LSTM RNN

In [ ]:
import pandas as pd
import numpy as np
import os
from pathlib import Path
from zipfile import ZipFile
import json

In [ ]:
!pip install kaggle

In [ ]:
with ZipFile('/content/train.csv.zip','r') as zipfile:
  zipfile.extractall()

In [ ]:
!ls

sample_data  train.csv	train.csv.zip


In [ ]:
df = pd.read_csv('/content/train.csv')

In [ ]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20800 entries, 0 to 20799
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      20800 non-null  int64 
 1   title   20242 non-null  object
 2   author  18843 non-null  object
 3   text    20761 non-null  object
 4   label   20800 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 812.6+ KB


In [ ]:
df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

### We will take the title of all the data and train the LSTM RNN to find out uf it is fake or not

In [ ]:
df = df.dropna()

In [ ]:
X = df.drop('label',axis=1)
y = df['label']

In [ ]:
X.shape

(18285, 4)

In [ ]:
y.shape

(18285,)

In [ ]:
!pip install tensorflow

In [ ]:
import tensorflow as tf

In [ ]:
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [ ]:
## Setting vocabulary size

voc_size = 5000

##### One Hot Representation

In [ ]:
messages = X.copy()

In [ ]:
messages.reset_index(inplace=True)

In [ ]:
messages

,index,id,title,author,text
0,0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...,...,...
18280,20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...
18281,20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...
18282,20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...
18283,20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal..."


In [ ]:
import nltk
import re
from nltk.corpus import stopwords

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
## Data preprocessing

from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()
corpus=[]

for i in range(0,len(messages)):
  review  = re.sub('[^a-zA-Z]', ' ' ,messages['title'][i])
  review = review.lower()
  review  = review.split()

  review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
  review = ' '.join(review)
  corpus.append(review)

In [ ]:
corpus

In [ ]:
corpus[1]

'flynn hillari clinton big woman campu breitbart'

In [ ]:
### One - Hot Representation

onehot_repr = [one_hot(word,voc_size) for word in corpus ]
onehot_repr

In [ ]:
onehot_repr[1]

[1461, 2920, 2231, 744, 561, 2654, 4745]

#### Embedding layer or Word to Vector

In [ ]:
### Padding to create an equal length input

In [ ]:
sent_length = 20

embedded_docs = pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
embedded_docs

array([[   0,    0,    0, ...,  424,  494, 2641],
       [   0,    0,    0, ...,  561, 2654, 4745],
       [   0,    0,    0, ..., 3025, 4385, 3666],
       ...,
       [   0,    0,    0, ..., 2052, 2218, 2794],
       [   0,    0,    0, ..., 1648, 2621, 4346],
       [   0,    0,    0, ..., 1852, 4872, 3868]], dtype=int32)

In [ ]:
embedded_docs[1]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0, 1461, 2920, 2231,  744,  561, 2654, 4745], dtype=int32)

In [ ]:
from tensorflow.keras.layers import Dropout

In [ ]:
### Creating model

embedding_vector_feature = 40 #each and every word is going to have 40 fearures
model = Sequential()
model.add(Embedding(voc_size,embedding_vector_feature,input_length=sent_length))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])


In [ ]:
print(model.summary())

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 20, 40)            200000    
                                                                 
 dropout (Dropout)           (None, 20, 40)            0         
                                                                 
 lstm_2 (LSTM)               (None, 100)               56400     
                                                                 
 dropout_1 (Dropout)         (None, 100)               0         
                                                                 
 dense_2 (Dense)             (None, 1)                 101       
                                                                 
Total params: 256501 (1001.96 KB)
Trainable params: 256501 (1001.96 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [ ]:
import numpy as np

In [ ]:
X_final = np.array(embedded_docs)

In [ ]:
y_final=np.array(y)

In [ ]:
X_final.shape

(18285, 20)

In [ ]:
y_final.shape

(18285,)

In [ ]:
y_final

array([1, 0, 1, ..., 0, 1, 1])

In [ ]:
## Train Test Split

from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test = train_test_split(X_final,y_final,test_size=0.30,random_state=0)

#### Model Training

In [ ]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
200/200 [==============================] - 14s 51ms/step - loss: 0.3480 - accuracy: 0.8346 - val_loss: 0.1990 - val_accuracy: 0.9149
Epoch 2/10
200/200 [==============================] - 12s 62ms/step - loss: 0.1519 - accuracy: 0.9402 - val_loss: 0.1847 - val_accuracy: 0.9200
Epoch 3/10
200/200 [==============================] - 13s 67ms/step - loss: 0.1163 - accuracy: 0.9564 - val_loss: 0.1982 - val_accuracy: 0.9176
Epoch 4/10
200/200 [==============================] - 15s 77ms/step - loss: 0.0920 - accuracy: 0.9669 - val_loss: 0.2205 - val_accuracy: 0.9165
Epoch 5/10
200/200 [==============================] - 18s 88ms/step - loss: 0.0735 - accuracy: 0.9741 - val_loss: 0.2899 - val_accuracy: 0.8994
Epoch 6/10
200/200 [==============================] - 21s 106ms/step - loss: 0.0589 - accuracy: 0.9798 - val_loss: 0.2530 - val_accuracy: 0.9085
Epoch 7/10
200/200 [==============================] - 11s 54ms/step - loss: 0.0453 - accuracy: 0.9849 - val_loss: 0.3182 - val_accuracy

In [ ]:
### PERFORMANCE METRICS

In [ ]:
y_pred = model.predict(X_test)

172/172 [==============================] - 3s 8ms/step


In [ ]:
y_pred=np.where(y_pred >0.5,1,0)  #whenever my y_pred is greater than 0.5, then put it as 1 or fake

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix,classification_report

In [ ]:
print(confusion_matrix(y_test,y_pred))

[[2810  274]
 [ 191 2211]]


In [ ]:
print(accuracy_score(y_test,y_pred))

0.9152387896463726


In [ ]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.94      0.91      0.92      3084
           1       0.89      0.92      0.90      2402

    accuracy                           0.92      5486
   macro avg       0.91      0.92      0.91      5486
weighted avg       0.92      0.92      0.92      5486

